In [ ]:
import os

import pandas as pd
from sqlalchemy import Column, Float, Integer, String, create_engine
from sqlalchemy.dialects.postgresql import insert  # It also works for SQLite for this basic usage
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Run this notebook on the directory supposed to store the project DB
path = os.getcwd()

engine = create_engine(f'sqlite:///{path}\\oldestbusinesses.db', echo=True)
Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base()


class Category(Base):
    __tablename__ = 'categories'
    category_code = Column(String(5), primary_key=True)
    category = Column(String(50))


class Country(Base):
    __tablename__ = 'countries'
    
    country_code = Column(String(3), primary_key=True)
    country = Column(String(50))
    continent = Column(String(20))


class Business(Base):

    __tablename__ = 'businesses'

    business = Column(String(64), primary_key=True)
    year_founded = Column(Integer)
    category_code = Column(String(5))
    country_code = Column(String(3))


def insert_data(table, data):
    query = insert(table).values(data)

    session.execute(query)
    session.commit()
    session.close()

Base.metadata.create_all(engine)

data_sets = {'categories': Category,
             'countries': Country,
             'businesses': Business}

for data, table in data_sets.items():
    raw_data = pd.read_csv(f'{data}.csv')
    data_as_dict = raw_data.to_dict('records')

    insert_data(table, data_as_dict)